# Load Video:

Load the provided video file.

In [24]:
import cv2

video_path = 'scenecutsample.mp4'

video_capture = cv2.VideoCapture(video_path)

video_loaded = video_capture.isOpened()

video_loaded


True

# Frame Extraction:

Extract individual frames from the video.

In [25]:

extracted_frames = []

frame_index = 0
while video_capture.isOpened():
    ret, frame = video_capture.read()
    
    if ret:
        extracted_frames.append((frame_index, frame))
        frame_index += 1
    else:
        break

video_capture.release()

len(extracted_frames)


638

# Spatio-Temporal Segmentation:

Perform segmentation on each frame using a technique like color thresholding or edge detection.\
Track the segmented objects across frames to observe changes in motion and shape.\
Identify the regions that remain consistent over time (foreground vs. background segmentation).

In [27]:
import cv2
import numpy as np

video_path = 'scenecutsample.mp4'
video_capture = cv2.VideoCapture(video_path)

frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_fg_mask = cv2.VideoWriter('foreground_mask_output.mp4', fourcc, fps, (frame_width, frame_height), isColor=False)
out_edges = cv2.VideoWriter('edges_output.mp4', fourcc, fps, (frame_width, frame_height), isColor=False)

ret, first_frame = video_capture.read()
gray_first_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
gray_first_frame = cv2.GaussianBlur(gray_first_frame, (5, 5), 0)

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    frame_diff = cv2.absdiff(gray_first_frame, gray_frame)
    _, fg_mask = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
    edges = cv2.Canny(fg_mask, 50, 150)

    kernel = np.ones((5, 5), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)

    out_fg_mask.write(fg_mask)
    out_edges.write(edges)

    cv2.imshow('Original Frame', frame)
    cv2.imshow('Foreground Mask', fg_mask)
    cv2.imshow('Edges', edges)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

video_capture.release()
out_fg_mask.release()
out_edges.release()
cv2.destroyAllWindows()


In [28]:
from skimage.metrics import structural_similarity as ssim
video_path = 'scenecutsample.mp4'
sample_image_path = 'targetimage.jpg' 
output_dir = 'output_frame'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

sample_image = cv2.imread(sample_image_path)
gray_sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2GRAY)

gray_sample_image = cv2.resize(gray_sample_image, (gray_sample_image.shape[1] // 2, gray_sample_image.shape[0] // 2))
sample_image_height, sample_image_width = gray_sample_image.shape

frame_counter = 0
while video_capture.isOpened():
    ret, color_frame = video_capture.read() 
    if not ret:
        break
    color_frame = cv2.resize(color_frame, (color_frame.shape[1] // 2, color_frame.shape[0] // 2))

    gray_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2GRAY)

    best_ssim = -1
    best_x, best_y = 0, 0
    step_size = 30

    for y in range(0, gray_frame.shape[0] - sample_image_height, step_size):  
        for x in range(0, gray_frame.shape[1] - sample_image_width, step_size):
            frame_region = gray_frame[y:y + sample_image_height, x:x + sample_image_width]

            similarity = ssim(frame_region, gray_sample_image)
            if similarity > best_ssim:
                best_ssim = similarity
                best_x, best_y = x, y

    cv2.rectangle(color_frame, (best_x, best_y), (best_x + sample_image_width, best_y + sample_image_height), (0, 255, 0), 2)
    output_frame_path = os.path.join(output_dir, f'frame_{frame_counter:04d}.jpg')
    cv2.imwrite(output_frame_path, color_frame, [int(cv2.IMWRITE_JPEG_QUALITY), 80])  
    
    frame_counter += 1

video_capture.release()
print(f"Frames saved in the folder: {output_dir}")

Frames saved in the folder: output_frame


# Scene Cut Detection:

Use pixel-based comparison or histogram differences between consecutive frames to detect abrupt changes (hard cuts).\
Detect gradual scene transitions (Soft cuts) by analyzing frame-to-frame intensity changes over time.

In [10]:
def detect_hard_cuts(frames, threshold=500000):
    cuts = []
    for i in range(1, len(frames)):
        diff = cv2.absdiff(frames[i], frames[i-1])
        non_zero_count = np.count_nonzero(diff)
        if non_zero_count > threshold:
            cuts.append(i)
    return cuts

In [13]:
def detect_soft_cuts(frames, hard_cuts, threshold=0.998):
    cuts = []
    for i in range(1, len(frames)):
        if i in hard_cuts:
            continue
        hist1 = cv2.calcHist([frames[i-1]], [0], None, [256], [0, 256])
        hist2 = cv2.calcHist([frames[i]], [0], None, [256], [0, 256])
        hist_diff = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
        if hist_diff < threshold:
            cuts.append(i)
    return cuts

In [34]:
import os
def mark_cuts(frames, hard_cuts, soft_cuts):
    marked_frames = []
    for i, frame in enumerate(frames):
        if i in hard_cuts:
            cv2.putText(frame, "Hard Cut", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            frame = cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 5)
        elif i in soft_cuts:
            cv2.putText(frame, "Soft Cut", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            frame = cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), 5)
        marked_frames.append(frame)
    return marked_frames



# Mark Scene Cuts:

Highlight the frames where scene cuts are detected.
Create a summary displaying the detected scene boundaries.

In [33]:
segmented_frames_folder = 'segmented_frames'
marked_frames_folder = 'marked_frames'
if not os.path.exists(marked_frames_folder):
    os.makedirs(marked_frames_folder)

frame_files = sorted([f for f in os.listdir(segmented_frames_folder) if f.endswith('.png')])

frames = []
for frame_file in frame_files:
    frame_path = os.path.join(segmented_frames_folder, frame_file)
    frame = cv2.imread(frame_path)
    if frame is None:
        print(f"Error reading frame {frame_file}")
        continue
    frames.append(frame)

hard_cuts = detect_hard_cuts(frames)
soft_cuts = detect_soft_cuts(frames, hard_cuts)
marked_frames = mark_cuts(frames, hard_cuts, soft_cuts)

In [35]:
for i, marked_frame in enumerate(marked_frames):
    frame_filename = os.path.join(marked_frames_folder, f'marked_frame_{i:04d}.png')
    cv2.imwrite(frame_filename, marked_frame)

print(f"Hard cuts detected at: {hard_cuts}")
print(f"Soft cuts detected at: {soft_cuts}")

Hard cuts detected at: [30, 48, 95, 429]
Soft cuts detected at: [66, 95, 302]


# Result Visualization:

Display frames where scene cuts are identified and show segmentation results for selected frames.

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt

segmented_frames_folder = 'segmented_frames'
frame_files = sorted([f for f in os.listdir(segmented_frames_folder) if f.endswith('.png')])

cut_indices = set(hard_cuts + soft_cuts)  
for i in cut_indices:
    if i < len(frame_files):
        frame_path = os.path.join(segmented_frames_folder, frame_files[i])
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"Error reading frame {frame_files[i]}")
            continue
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 6))
        plt.imshow(frame_rgb)
        if i in hard_cuts and i in soft_cuts:
            plt.title(f"Hard and Soft Cut at Frame {i}")
        elif i in hard_cuts:
            plt.title(f"Hard Cut at Frame {i}")
        elif i in soft_cuts:
            plt.title(f"Soft Cut at Frame {i}")
        
        plt.axis('off') 
        plt.show()
    else:
        print(f"Frame index {i} is out of bounds for the available frames.")
